<img width=200 height=250 align=center src="logouniversidad.png">
 
<h3 align="center">                                              Bases de Datos II</h3> 

Alumnos:
* Oscar Rene Soto Sánchez 
* Juan Carlos Rojas Coffey 
* Jesús Armando Báez Camacho


# Ventas Internacionales en Sonora (2021)

<p style="text-align: justify;"> Este proyecto busca contrastar el tipo de ventas y comercio realizado en el Estado de Sonora durante el año pasado, justo despues de salir de una pandemia que duro 2 años, y contrastar los datos de las ventas con los estados del pais con economias mas fuertes. </p>

La base de datos utilizada la encontramos en datamexico.org.

Se pretenden utilizar 4 tablas de datos diferentes, de 4 estados de la republica que representan las ventas internacionales realizadas en el periodo de 2021. 

* Sonora:https://datamexico.org/es/profile/geo/sonora-so
* Nuevo Leon: https://datamexico.org/es/profile/geo/nuevo-leon-nl
* Ciudad de Mexico: https://datamexico.org/es/profile/geo/ciudad-de-mexico-cx
* Jalisco:https://datamexico.org/es/profile/geo/jalisco-jc

De las 4 tablas construimos una sola, para metodos practicos, con la ayuda de la herramienta VizBuilder que ofrece la pagina de DataMexico.

* Fuente: https://datamexico.org/es/vizbuilder

In [53]:
import mysql.connector
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="productosinternacionales"
)

query = "SELECT * FROM productos2021csv;"
result_dataFrame = pd.read_sql(query,mydb)
df

,Flow ID,Flow,HS4 4 Digit ID,HS4 4 Digit,Date Year,State ID,State,Chapter 4 Digit ID,Chapter 4 Digit,Trade Value
0,2,Exports,10101,"Horses, Asses, Mules and Hinnies",2021,26,Sonora,1,Animal Products,85204
1,2,Exports,10102,Live Animals of the Bovine Species,2021,9,Ciudad de México,1,Animal Products,2858060
2,2,Exports,10102,Live Animals of the Bovine Species,2021,14,Jalisco,1,Animal Products,412094
3,2,Exports,10102,Live Animals of the Bovine Species,2021,19,Nuevo León,1,Animal Products,6916241
4,2,Exports,10102,Live Animals of the Bovine Species,2021,26,Sonora,1,Animal Products,87167022
...,...,...,...,...,...,...,...,...,...,...
2628,2,Exports,219701,"Paintings and Drawings, Executed Entirely by H...",2021,14,Jalisco,21,Arts and Antiques,320528
2629,2,Exports,219701,"Paintings and Drawings, Executed Entirely by H...",2021,19,Nuevo León,21,Arts and Antiques,65148
2630,2,Exports,219702,"Engravings, Prints and Lithographs",2021,9,Ciudad de México,21,Arts and Antiques,15761
2631,2,Exports,219702,"Engravings, Prints and Lithographs",2021,14,Jalisco,21,Arts and Antiques,19016


# Nombres, tipo y descripcion de cada variable de la tabla usada.

El *tipo* esta en cursiva, y el **nombre de la variable** en negritas.

* *INT* **Flow ID**: Este parametro define que tipo de flujo se realizó. Sus valores son 1 o 2.
* *STRING* **Flow**: Es el termino concreto del flujo, importacion o exportacion. Coindicen con los valores Flow ID respectivamente.
* *INT* **HS4 4 Digit ID**: Este parametro define, según el nivel del producto, a que area pertenece cada producto, usan 4 digitos. 
* *STRING* **HS4 4 Digit**: Es un parametro que detalla en mayor medida lo que el parametro chapter.
* *DATE* **Date Year**: Es el año en el cual se hizo la transaccion.
* *INT* **State ID**: Es el numero asignado a cada estado, numeros alfabeticamente, incluyendo la Ciudad de Mexico.
* *STRING* **State**: Es el nombre del estado como texto.
* *INT* **Chapter 4 Digit ID**: Este parametro define que tipo de producto se comercializa, adquiere valores del 1 al 21.
* *STRING* **Chapter 4 Digit**: Es el tipo de producto que se comercializa, según el nivel del producto. 
* *INT* **Trade Value**: El valor neto en dolares generado por la transaccion.

A continuacion esta la tabla del Diagrama Entidad-Relacion generado al final de la consulta:

<img width=680 height=540 align=center src="DiagramaER.png">

In [44]:
df.columns

Index(['Flow ID', 'Flow', 'HS4 4 Digit ID', 'HS4 4 Digit', 'Date Year',
       'State ID', 'State', 'Chapter 4 Digit ID', 'Chapter 4 Digit',
       'Trade Value'],
      dtype='object')

# Consultas
### Funcion en Python usando QUERYS de MySQL

#### Ejemplo 1
En la siguiente función obtendremos la ganancia total dependiendo del tipo de producto comercializado segun el estado, adjunto con un ejemplo:

In [54]:
def GanaciasTotalesPorProducto(Estado, Producto):
    query = "SELECT SUM(`Trade Value`) AS Ganancias FROM productos2021csv WHERE (`Chapter 4 Digit` = '" + Producto + "') AND (State = '"+ Estado +"');"
    dfq = pd.read_sql(query,mydb)
    print(dfq) 

In [55]:
GanaciasTotalesPorProducto("Sonora", "Animal Products");

     Ganancias
0  774293916.0


#### Ejemplo 2
En este ejemplo filtramos el valor maximo obtenido por las ventas de cada Estado.

In [60]:
def GananciasTotalesPorEstado(Estado):
    query = "SELECT SUM(`Trade Value`) AS Ganancia FROM productos2021csv WHERE (State = '"+ Estado +"');"
    dfq = pd.read_sql(query,mydb)
    print(dfq)


In [58]:
GananciasTotalesPorEstado('Nuevo León');

       Ganancia
0  4.703501e+10


#### Ejemplo 3 y 4
En este ejemplo, se da como valor el Estado que se quiere buscar, y te regresa el producto exportado de mayor valor y a que area pertenece. 

Encontramos una anomalia en los datos. Para el valor maximo de Jalisco, Ciudad de México y Nuevo León, el tipo de dato no alcanza para la cifra que se espera de los datos de la Secretaria de Comercio.

Tambien se puede cambiar la funcion SELECT MAX del QUERY por un SELECT MIN, y se pueden visualizar los diferentes tipos de datos de la misma manera.

In [71]:
def GananciaMaximaPorProducto(Estado):
    query = "SELECT `HS4 4 Digit`, `Trade Value`,`Chapter 4 Digit` FROM productos2021csv WHERE (`Trade Value` = (SELECT MAX(`Trade Value`) FROM productos2021csv WHERE State = '"+ Estado +"'));"
    dfq = pd.read_sql(query,mydb)
    print(dfq)

In [75]:
GananciaMaximaPorProducto('Sonora')

                   HS4 4 Digit  Trade Value Chapter 4 Digit
0  Electrical Wires and Cables   1547839483        Machines


In [76]:
def GananciaMinimaPorProducto(Estado):
    query = "SELECT `HS4 4 Digit`, `Trade Value`,`Chapter 4 Digit` FROM productos2021csv WHERE (`Trade Value` = (SELECT MIN(`Trade Value`) FROM productos2021csv WHERE State = '"+ Estado +"'));"
    dfq = pd.read_sql(query,mydb)
    print(dfq)

In [80]:
GananciaMinimaPorProducto('Jalisco')

                                         HS4 4 Digit  Trade Value  \
0  Time Control Devices and Timers, with Clockwor...          299   

  Chapter 4 Digit  
0     Instruments  


#### Ejemplo 5
Calculamos la ganancia promedio por Estado y Producto.

In [89]:
def GanaciasPromedioPorProducto(Estado, Producto):
    query = "SELECT AVG(`Trade Value`) AS GananciasPromedio FROM productos2021csv WHERE (`Chapter 4 Digit` = '" + Producto + "') AND (State = '"+ Estado +"');"
    dfq = pd.read_sql(query,mydb)
    print(dfq)

In [88]:
GanaciasPromedioPorProducto('Ciudad de México', 'Mineral Products')

     Ganancias
0  156653915.5


# Referencias
* Base de Datos: https://datamexico.org/